<a href="https://colab.research.google.com/github/ANKITPODDER2000/CNN_Colab/blob/master/TextSequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sen_set = [
  "I am Ankit Podder",
  "I am ankit",
  "I love my cat!"
]
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 1000 , oov_token = "<OOV>")
tokenizer.fit_on_texts(sen_set)
seq =  tokenizer.texts_to_sequences(sen_set)
mat = tokenizer.texts_to_matrix(sen_set , mode = "binary")
print("Seq := ",seq)
print("Mat := ",mat)

In [ ]:
tokenizer.texts_to_matrix(["I am ankit"]).shape

In [ ]:
tokenizer.word_index

In [ ]:
tokenizer.word_counts

In [ ]:
tokenizer.index_word

In [ ]:
import os
os.chdir("/content/drive/My Drive/NLP/data")

In [ ]:
import pandas as pd

msg = pd.read_csv("./"+os.listdir()[0])

In [ ]:
msg.head()

In [ ]:
def ch_category(x):
    if x=='spam':
        return 1
    return 0

In [ ]:
msg['Category'] = msg['Category'].apply(ch_category)

In [ ]:
msg.head()

In [ ]:
max_len_msg = max([len(i) for i in msg['Message']])
print("Max len of msg is := ",max_len_msg)

In [ ]:
num_words  = 10000
emb_dim    = 32
maxlen     = 120
truncating = "post"
padding    = "post"
oov_token  = "<OOV>"

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(msg['Message'] , msg['Category'] , test_size = 0.2)

In [ ]:
train_data = []
train_label = []

test_data = []
test_label = []

In [ ]:
for dat , lab in zip(X_train , y_train):
  train_data.append(dat)
  train_label.append(lab)

for dat , lab in zip(X_test , y_test):
  test_data.append(dat)
  test_label.append(lab)

In [ ]:
print(len(train_data))
print(len(train_label))
print(len(test_data))
print(len(test_label))

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = num_words , oov_token = oov_token)
tokenizer.fit_on_texts(train_data)
train_seq = tokenizer.texts_to_sequences(train_data)
test_seq  = tokenizer.texts_to_sequences(test_data )

In [ ]:
train_pad = tf.keras.preprocessing.sequence.pad_sequences(train_seq , maxlen = maxlen , padding = padding , truncating = truncating)
test_pad  = tf.keras.preprocessing.sequence.pad_sequences(test_seq  , maxlen = maxlen , padding = padding , truncating = truncating)

In [ ]:
import numpy as np
train_label = np.array(train_label)
test_label  = np.array(test_label )

In [ ]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words , emb_dim , input_length = maxlen),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64 , activation = "relu"),
    tf.keras.layers.Dense(1 , activation = "sigmoid")
])
model1.compile(optimizer = "rmsprop" , loss = tf.keras.losses.binary_crossentropy , metrics = ['acc'])
model1.summary()

In [ ]:
his1 = model1.fit(train_pad , train_label , epochs = 50 , validation_data = (test_pad , test_label))

In [ ]:
def his_details(history):
    sns.set_style("whitegrid")
    _ , ax = plt.subplots(1 , 2 , figsize = (24,6))
    
    ax[0].plot(history.history['acc'] , label = "train data")
    ax[0].plot(history.history['val_acc'] , label = "test data")
    ax[0].set_title("Accurecy")
    ax[0].legend()

    ax[1].plot(history.history['loss'] , label = "train data")
    ax[1].plot(history.history['val_loss'] , label = "test data")
    ax[1].set_title("Crossentropy")
    ax[1].legend()

In [ ]:
his_details(his1)

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words , emb_dim , input_length = maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(64 , activation = "relu"),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64 , activation = "relu"),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1 , activation = "sigmoid")
])
model2.compile(optimizer = "rmsprop" , loss = tf.keras.losses.binary_crossentropy , metrics = ['acc'])
model2.summary()
his1 = model2.fit(train_pad , train_label , epochs = 25 , validation_data = (test_pad , test_label))

In [ ]:
his_details(his1)#this is for model2

In [ ]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words , emb_dim , input_length = maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64 , activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64 , activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1 , activation = "sigmoid")
])
model3.compile(optimizer = "rmsprop" , loss = tf.keras.losses.binary_crossentropy , metrics = ['acc'])
model3.summary()
his3 = model3.fit(train_pad , train_label , epochs = 25 , validation_data = (test_pad , test_label))

In [ ]:
his_details(his3)#this is for model3

In [ ]:
model4 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words , emb_dim , input_length = maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64 , kernel_regularizer = tf.keras.regularizers.l2(0.0001) ,  activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64 , kernel_regularizer = tf.keras.regularizers.l2(0.0001) ,  activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1 , activation = "sigmoid")
])
model4.compile(optimizer = "rmsprop" , loss = tf.keras.losses.binary_crossentropy , metrics = ['acc'])
model4.summary()
his4 = model4.fit(train_pad , train_label , epochs = 20 , batch_size = 32 , verbose = 0 , validation_data = (test_pad , test_label))
his_details(his4)#this is for model4

In [ ]:
def seq_to_str(seq):
    s = ""
    for i in seq:
        if i==0 :
            break
        s += tokenizer.index_word[i] + " "
    s = s[:-1]
    s += "."
    return s

In [ ]:
seq_to_str(test_pad[0 , :])

In [ ]:
test_label[0:10]

In [ ]:
model3.predict_classes(test_pad[:10])[:,0]

In [ ]:
def pred(s , model = model3):
  classes = ["Ham" , "Spam"]
  str_test = [s]
  seq_check = tokenizer.texts_to_sequences(str_test)
  #print(seq_check)
  seq_check = tf.keras.preprocessing.sequence.pad_sequences(seq_check , maxlen = maxlen , padding = padding , truncating = truncating)
  print("%s -> %s"%(str_test[0] , classes[model3.predict_classes(seq_check)[0,0]]))

In [ ]:
pred("You won a huge money prize , plz claim your money!")

In [ ]:
pred("Piyal Sarkar posted a new announcement in MC-IT501.")

In [ ]:
pred("A nostalgia we all love to delve in, September 5 brings back memories while we reminisce about our school days. From the light-speed quick “Happy Teachers’ Day!”, to gifting them stationery and greeting cards, this day holds a special place in our hearts.But how often have we really thought about that one teacher who changed our lives in ways we now cherish and remember fondly?Well, after all these years, it's now time to acknowledge those teachers. And this time, GeeksforGeeks has the nicest way for you to tell us about that one teacher who was nothing less than an inspiration to you.The best accounts shall get featured in our monthly newsletter, Geeks Digest as well as on our social media handles. Fill in the form below to share your story with the world!")

In [ ]:
pred("You were added to this channel on Aug 28, 2020")

In [ ]:
pred("Hi friends")

In [ ]:
pred("LAST DAY TODAY!! Samsung Big Campus Sale!!")